In [1]:
from pydlr import kernel, dlr
from triqs.gf import *
from h5 import HDFArchive
import numpy as np
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

import matplotlib.pyplot as plt
plt.style.use('publish')

from dimer import *
from common import *

Starting serial run at: 2023-02-10 16:17:37.048414


In [2]:
sigma_moments = sigma_high_frequency_moments(dm, hdiag, gf_struct, h_int)
sig_infty, sigma_1 = sigma_moments['up'][0], sigma_moments['up'][1]

In [3]:
d = dlr(lamb=30)
freq = d.get_matsubara_frequencies(beta)
G0_tau = make_gf_from_fourier(G0_iw)
tau_i = np.array([x.real for x in G0_tau.mesh])
iw_i  = np.array([complex(x) for x in Sigma_iw_ref.mesh])

In [4]:
g_iaa = G_tau_ref['up'].data
g0_iaa = G0_tau['up'].data
g_xaa = d.lstsq_dlr_from_tau(tau_i, g_iaa, beta)
g0_xaa = d.lstsq_dlr_from_tau(tau_i, g0_iaa, beta)

In [5]:
gk_iwaa = d.matsubara_from_dlr(g_xaa, beta)
g0k_iwaa = d.matsubara_from_dlr(g0_xaa, beta)

In [6]:
σk_iwaa = np.linalg.inv(g0k_iwaa[:,...])-np.linalg.inv(gk_iwaa[:,...])

In [102]:
rk_iwaa = gk_iwaa - g0k_iwaa[:,...]@σk_iwaa[:,...]@gk_iwaa[:,...]  - g0k_iwaa

In [103]:
r_xaa = d.lstsq_dlr_from_matsubara(freq, rk_iwaa, beta)

In [117]:
Mkl = np.zeros((len(d), len(d)))
for iwk, wk in enumerate(d.dlrrf):
    for iwl, wl in enumerate(d.dlrrf):
        K0wk, Kbwk = kernel(np.array([0,1]), np.array([wk])).flatten()
        K0wl, Kbwl = kernel(np.array([0, 1]), np.array([wl])).flatten()
        if np.fabs(wk+wl) < 1e-13:
            Mkl[iwk,iwl] = K0wk*K0wl
        else:
            Mkl[iwk, iwl] = (1/(beta*(wk+wl)))*(K0wk*K0wl - Kbwk*Kbwl)

In [118]:
np.linalg.norm(np.einsum('nmk,kl,lnm->nm', r_xaa.T, Mkl, r_xaa).flatten())

3.636843318108796e-15

In [111]:
sym = Symmetrizer(len(d), 2)
nx, no = len(d), 2
dtype=complex
def merge_re_im(x):
    x_d, x_u = x[:nx*no], x[nx*no:]
    re, im = np.split(x_u, 2)
    x_u = re + 1.j * im
    return x_d, x_u

def split_re_im(x_d, x_u):
    return np.concatenate((
            np.array(x_d.real, dtype=float),
            np.array(x_u.real, dtype=float),
            np.array(x_u.imag, dtype=float)))
def g_from_x(x):
    x_d, x_u = merge_re_im(x)
    g_xaa = np.zeros((nx, no, no), dtype=dtype)
    sym.set_x_u(g_xaa, x_u)
    sym.set_x_d(g_xaa, x_d)
    return g_xaa
                        
def x_from_g(g_xaa):
    x_d = sym.get_x_d(g_xaa)
    x_u = sym.get_x_u(g_xaa)
    x = split_re_im(x_d, x_u)
    return x

In [12]:
-sig.sum(axis=0)-sigma_1

array([[1.22094548e-04+0.00039095j, 6.31444966e-05+0.00023679j],
       [5.94477405e-05+0.00024739j, 8.33536698e-05+0.00041058j]])

In [13]:
class Symmetrizer:

    def __init__(self, nx, no):
        self.N = (no*(no-1))//2
        self.nx, self.no = nx, no
        self.diag_idxs = np.arange(self.no)
        self.triu_idxs = np.triu_indices(no, k=1)
        self.tril_idxs = np.tril_indices(no, k=-1)
    
    def get_x_d(self, g_xaa):
        x_d = g_xaa[:, self.diag_idxs, self.diag_idxs].flatten()
        return x_d

    def set_x_d(self, g_xaa, x_d):
        g_xaa[:, self.diag_idxs, self.diag_idxs] = x_d.reshape((self.nx, self.no))
        return g_xaa

    def get_x_u(self, g_xaa):
        x_u = g_xaa[:, self.triu_idxs[0], self.triu_idxs[1]].flatten()
        return x_u

    def set_x_u(self, g_xaa, x_u):
        g_xaa[:, self.triu_idxs[0], self.triu_idxs[1]] = x_u.reshape((self.nx, self.N))
        g_xaa[:, self.tril_idxs[0], self.tril_idxs[1]] = g_xaa[:, self.triu_idxs[0], self.triu_idxs[1]].conj()
        #g_xaa += np.transpose(g_xaa, axes=(0,2,1)).conj()
        return g_xaa

    def get_diag_indices(self): return self.diag_idxs
    def get_triu_indices(self): return self.triu_idxs

In [194]:
def constrained_lstsq_dlr_from_tau(d,         # dlr class
                                   tau_i,     # tau mesh
                                   beta,      # inverse temperature
                                   g_iaa,     # G data
                                   g0_iaa,    # G0 data
                                   sigma_moments, # high-freq moments of Σ
                                   ftol=1e-9, 
                                   symmetrizer=None):
    
    
    
    
    
    nx = len(d)
    ni, no, _ = g_iaa.shape
    shape_xaa = (nx, no, no)
    N = (no*(no-1))//2

    dtype = complex
    nX = nx * (no + 2*N)
    
    # precompute Mkl
    def compute_mkl(d):
        Mkl = np.zeros((len(d), len(d)))
        for iwk, wk in enumerate(d.dlrrf):
            for iwl, wl in enumerate(d.dlrrf):
                K0wk, Kbwk = kernel(np.array([0,1]), np.array([wk])).flatten()
                K0wl, Kbwl = kernel(np.array([0,1]), np.array([wl])).flatten()
                if np.fabs(wk+wl) < 1e-13:
                    Mkl[iwk,iwl] = K0wk*K0wl
                else:
                    Mkl[iwk, iwl] = (1/(beta*(wk+wl)))*(K0wk*K0wl - Kbwk*Kbwl)
        return Mkl
    
    # fold and unfold complex numbers
    
    def merge_re_im(x):
        x_d, x_u = x[:nx*no], x[nx*no:]
        re, im = np.split(x_u, 2)
        x_u = re + 1.j * im
        return x_d, x_u

    def split_re_im(x_d, x_u):
        return np.concatenate((
            np.array(x_d.real, dtype=float),
            np.array(x_u.real, dtype=float),
            np.array(x_u.imag, dtype=float)))
                                   
    # Greens function <-> vector conversion

    sym = symmetrizer if symmetrizer is not None else Symmetrizer(nx, no)
    
    def g_from_x(x):
        x_d, x_u = merge_re_im(x)
        g_xaa = np.zeros((nx, no, no), dtype=dtype)
        sym.set_x_u(g_xaa, x_u)
        sym.set_x_d(g_xaa, x_d)
        return g_xaa
                        
    def x_from_g(g_xaa):
        x_d = sym.get_x_d(g_xaa)
        x_u = sym.get_x_u(g_xaa)
        x = split_re_im(x_d, x_u)
        return x
    
    
    # constraint
    sig_infty, sigma_1 = sigma_moments[0], sigma_moments[1]
        
    def mat_vec(mat):
        v_d = sym.get_x_d(mat[None, ...]).real
        v_u = sym.get_x_u(mat[None, ...])
        v = split_re_im(v_d, v_u)
        return v
        
    def constraint_func(x):
        sig = g_from_x(x)
        sig_xaa = d.lstsq_dlr_from_matsubara(freq, sig-sig_infty, beta)
        mat = -sig_xaa.sum(axis=0)
        vec = mat_vec(mat)
        return vec
    
    bound = mat_vec(sigma_1)
    
    constraints = (NonlinearConstraint(constraint_func,
                                           bound, bound))


    # target function
    def dyson_difference(sigk):
        sigk_iwaa = g_from_x(sigk)
        #  G - G0 - G0*Σ*G = 0 done on the DLR nodes
        rk_iwaa = gk_iwaa - g0k_iwaa[:,...]@sigk_iwaa[:,...]@gk_iwaa[:,...]  - g0k_iwaa
        # compute DLR of rk_iwaa
        r_xaa = d.lstsq_dlr_from_matsubara(freq, rk_iwaa, beta)
        # ||R||^2 = r^T @ M @ r
        R = np.einsum('nmk,kl,lnm->nm', r_xaa.T, Mkl, r_xaa).flatten()
        return R
        
    def target_function(x):
        y = dyson_difference(x)
        return np.linalg.norm(y)
        
    
    # compute Mkl
    Mkl = compute_mkl(d)
    freq = d.get_matsubara_frequencies(beta)
    
    # dlr fit to G and G0
    g_xaa = d.lstsq_dlr_from_tau(tau_i, g_iaa, beta)
    g0_xaa = d.lstsq_dlr_from_tau(tau_i, g0_iaa, beta)
    
    # compute and obtain initial Σ
    gk_iwaa = d.matsubara_from_dlr(g_xaa, beta)
    g0k_iwaa = d.matsubara_from_dlr(g0_xaa, beta)
    sigk_iwaa = np.linalg.inv(g0k_iwaa[:,...])-np.linalg.inv(gk_iwaa[:,...])
    
    x_init = x_from_g(sigk_iwaa)

    # minimize
    sol = minimize(target_function, 
                   x_init,
        method='SLSQP', 
        constraints=constraints,
        options=dict(ftol=ftol, maxiter=1000, disp=True),
        )
    print(sol.success, sol.message)
    
    sigk_iwaa = g_from_x(sol.x)
    
    sig_xaa = d.lstsq_dlr_from_matsubara(freq, sigk_iwaa-sig_infty, beta)
    print(f'Σ1 constraint diff: {np.max(np.abs(-sig_xaa.sum(axis=0)-sigma_1)):.4e}')

    return sig_xaa, sol

In [195]:
G0_tau = make_gf_from_fourier(G0_iw)
tau_i = np.array([x.real for x in G0_tau.mesh])
iw_i  = np.array([complex(x) for x in Sigma_iw_ref.mesh])

In [ ]:
tol = 1e-4
G_tau_qmc= G_tau_ref.copy()
for block, _ in G_tau_qmc: G_tau_qmc[block].data[:] += np.random.normal(scale=tol, size=G_tau_qmc[block].data.shape)
d = dlr(lamb=10,eps=1e-6)

sig_xaa, sol = constrained_lstsq_dlr_from_tau(d, 
                                   tau_i, 
                                   beta,
                                   G_tau_qmc['up'].data,
                                   G0_tau['up'].data,
                                   sigma_moments['up'],
                                   ftol=1e-9, 
                                   symmetrizer=None)
Sigma_iw_fit = Sigma_iw_ref.copy()
Sigma_iw_fit['up'].data[:] = d.eval_dlr_freq(sig_xaa, iw_i, beta) +sigma_moments['up'][0]

In [ ]:
iwn = np.array([complex(x) for x in Sigma_iw_ref.mesh])
fig, ax = plt.subplots(2,2,sharex=True,)
ax[0,0].plot(iwn.imag, Sigma_iw_ref['up'].data[:,0,0].real, label='exact')
ax[0,0].plot(iwn.imag, Sigma_iw_fit['up'].data[:,0,0].real, label='fit')
ax[0,0].set_ylabel('ReΣ')
ax[1,0].semilogy(iwn.imag, np.abs(Sigma_iw_ref['up'].data[:,0,0].real-Sigma_iw_fit['up'].data[:,0,0].real))
ax[1,0].set_ylabel('ΔReΣ')
ax[1,0].set_xlabel('iωn')
ax[0,0].legend()
ax[0,1].plot(iwn.imag, Sigma_iw_ref['up'].data[:,0,0].imag, label='exact')
ax[0,1].plot(iwn.imag, Sigma_iw_fit['up'].data[:,0,0].imag, label='fit')
ax[0,1].set_ylabel('ImΣ')
ax[1,1].semilogy(iwn.imag, np.abs(Sigma_iw_ref['up'].data[:,0,0].imag-Sigma_iw_fit['up'].data[:,0,0].imag))
ax[1,1].set_ylabel('ΔImΣ')
ax[1,1].set_xlabel('iωn')
ax[0,0].legend()
ax[0,0].set_xlim(0,60)
plt.subplots_adjust(hspace=0.25, wspace=0.5)
plt.show()